In [1]:
import json
import numpy as np
import pandas as pd

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

In [2]:
pd.set_option('display.max_colwidth', 800)

In [3]:
sample_size = 1000
offset = 0
area_thresh = 1e5**2

## COCO

### Inspect original val2017 GT json

In [4]:
with open('./instances_val2017.json', 'r') as f:
    ann_dict = json.load(f)

In [5]:
assert len(ann_dict['images']) == 5000

In [6]:
assert len(ann_dict['annotations']) == 36781

### Remove unnecessary key value pairs

In [7]:
ann_dict.keys()

dict_keys(['info', 'licenses', 'images', 'annotations', 'categories'])

From top level.

In [8]:
remove_base = ['info', 'licenses']

In [9]:
for key in remove_base:
    del ann_dict[key]

In [10]:
ann_dict.keys()

dict_keys(['images', 'annotations', 'categories'])

For 'images' sub-dict, keep only 'id'.

In [11]:
remove_images = ['license', 'file_name', 'coco_url', 'height', 'width', 'date_captured', 'flickr_url']

In [12]:
for key in remove_images:
    for item in ann_dict['images']:
        del item[key]

In [13]:
ann_dict['images'][0]

{'id': 397133}

For 'annotations' sub-dict

In [14]:
remove_anno = ['segmentation']

In [15]:
for key in remove_anno:
    for item in ann_dict['annotations']:
        del item[key]

For 'categories' sub-dict

In [16]:
for item in ann_dict['categories']:
    del item['supercategory']

### Create small GT set

Get (sample_size) GT.

In [17]:
small_ann_dict = ann_dict['annotations'][0 + offset:sample_size + offset]
ann_dict['annotations'] = small_ann_dict
assert len(ann_dict['annotations']) == sample_size

From GT, we collect all the images involved.

In [18]:
image_ids = sorted(set([item['image_id'] for item in small_ann_dict[0:sample_size]]))
num_images = len(image_ids)
print(f'{num_images} images used.')
df = pd.DataFrame(ann_dict['images'])
small_images = df[df['id'].isin(image_ids)].to_dict('records')

325 images used.


In [19]:
ann_dict['images'] = small_images

In [20]:
with open('./test.json', 'w') as f:
    json.dump(ann_dict, f)

### Load GT

In [21]:
# annFile = './instances_val2017.json'
annFile = './test.json'
cocoGt=COCO(annFile)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [22]:
all_gt_id = list(cocoGt.anns.keys())
# assert len(all_id) == 36781
assert len(all_gt_id) == sample_size

In [23]:
all_gt = cocoGt.loadAnns(all_gt_id)

In [24]:
gt_df = pd.DataFrame(all_gt)

In [25]:
# every gt box has its own unique id
assert not gt_df['id'].duplicated().any()

In [26]:
# image_id can be duplicated, which means the boxes are in one iamge
gt_df[gt_df['image_id'] == 78823]

,area,iscrowd,image_id,bbox,category_id,id
7,16694.40470,0,78823,"[197.97, 117.22, 170.45, 222.07]",18,7544
29,9473.41725,0,78823,"[58.45, 81.69, 131.53, 125.9]",64,21771
372,91251.48390,0,78823,"[9.29, 160.0, 466.58, 314.84]",3,133097
379,26292.74015,0,78823,"[1.29, 150.59, 186.23, 261.24]",3,134130
447,4064.22540,0,78823,"[594.08, 175.0, 45.2, 128.07]",3,145780


### Generate fake pred and save as json

In [27]:
anns = cocoGt.loadAnns(all_gt_id[0:sample_size])

In [28]:
fake_pred = []
np.random.seed(seed=256)
scores = np.random.random((sample_size,)) 

for i, annotation in enumerate(anns):
    fake_pred.append({
        'image_id': annotation['image_id'],
        'category_id': annotation['category_id'],
        'bbox': (annotation['bbox'] + 2 * ((1 - (-1)) * np.random.random(4) + 1)).tolist(),
        'score': scores[i]
    })

In [29]:
fake_pred_df = pd.DataFrame(fake_pred).sort_values('image_id').reset_index()

In [30]:
assert len(fake_pred_df) == sample_size
assert len(fake_pred_df['image_id'].drop_duplicates()) == num_images

In [31]:
with open('fake_pred.json', 'wt') as f:
    json.dump(fake_pred, f)

### mAP using fake pred

In [32]:
assert len(cocoGt.anns) == sample_size

In [33]:
for gt_id in cocoGt.anns.keys():
    assert cocoGt.anns[gt_id]['iscrowd'] == 0

In [34]:
for gt_id in cocoGt.anns.keys():
    assert cocoGt.anns[gt_id]['area'] < area_thresh
    cocoGt.anns[gt_id]['area'] == 0

In [35]:
#initialize COCO detections api
cocoDt=cocoGt.loadRes('./fake_pred.json')

Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!


In [36]:
# running evaluation
cocoEval = COCOeval(cocoGt, cocoDt, 'bbox')
cocoEval.params.imgIds = image_ids
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()

Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.33s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.45150790883927888197391098401567
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.79817127849120594884624324549804
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.45887468496425481978562288531975


In [37]:
for gt_id in cocoGt.anns.keys():
    assert len(cocoGt.anns[gt_id].keys()) == 8
    assert cocoGt.anns[gt_id]['ignore'] == 0
    assert cocoGt.anns[gt_id]['_ignore'] == 0

In [38]:
# sample_size = 300
# offset = 1

# Running per image evaluation...
# Evaluate annotation type *bbox*
# DONE (t=0.09s).
# Accumulating evaluation results...
# DONE (t=0.01s).
#  Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.49353108110035337485754780573188
#  Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.83751546185462277804845143691637
#  Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.54370192869205802299603647043114

## FE mAP

### Create bbox

#### small set

In [39]:
df = pd.DataFrame(anns).sort_values('image_id').reset_index()

In [40]:
max_num_bbox = df['image_id'].value_counts().values[0]

In [41]:
bbox_shape = (num_images, max_num_bbox, 5)
bbox = np.zeros(bbox_shape)

In [42]:
for i, image_id in enumerate(image_ids):
    temp = df[df['image_id'].isin([image_id])][['bbox', 'category_id']]
    bbox[i, 0:len(temp), 0:4] = temp['bbox'].tolist()
    bbox[i, 0:len(temp), 4] = temp['category_id'].values

#### full set

### Create pred

In [43]:
pred = []
for image_id in image_ids:
    temp = fake_pred_df[fake_pred_df['image_id'].isin([image_id])][['bbox', 'category_id', 'score']]
    array = np.zeros((len(temp), 6))
    array[0:len(temp), 0:4] = temp['bbox'].tolist()
    array[0:len(temp), 4] = temp['category_id'].values
    array[0:len(temp), 5] = temp['score'].values
    pred.append(array)

In [44]:
pred[1]

array([[4.51028207e+00, 1.73676506e+02, 1.06509383e+02, 2.81414730e+02,
        8.20000000e+01, 2.66857900e-01],
       [5.04532244e+02, 3.11822834e+02, 3.48458784e+01, 8.74090115e+01,
        4.40000000e+01, 2.80023998e-01],
       [4.85535423e+02, 3.04165487e+02, 3.11288757e+01, 8.65428310e+01,
        4.40000000e+01, 4.52136994e-01]])

In [45]:
fake_pred_df.loc[1:2]

,index,image_id,category_id,bbox,score
1,756,7386,4,"[54.69798630073991, 17.495454049599488, 553.4570705725084, 391.49672393427124]",0.415159
2,910,7574,82,"[4.5102820696945365, 173.67650600970396, 106.50938308264257, 281.4147304286105]",0.266858


### Calculate mAP

In [46]:
from fastestimator.trace.metric import MeanAveragePrecision

In [47]:
mAP = MeanAveragePrecision(num_classes=90)

In [48]:
data = {}
data['pred'] = pred
data['bbox'] = bbox

In [49]:
mAP.on_epoch_begin(data)

In [50]:
mAP.on_batch_begin(data)

In [51]:
mAP.on_batch_end(data)

In [52]:
mAP.on_epoch_end(data)

In [53]:
data.keys()

dict_keys(['pred', 'bbox', 'mAP', 'AP50', 'AP75'])

In [54]:
mAP = data['mAP']
print(f"{mAP:.32f}")

0.45150790883927888197391098401567


In [55]:
ap50 = data['AP50']
print(f"{ap50:.32f}")

0.79817127849120594884624324549804


In [56]:
ap75 = data['AP75']
print(f"{ap75:.32f}")

0.45887468496425481978562288531975
